# Citation to Original Paper
Xiong Liu, Yu Chen, Jay Bae, Hu Li, Joseph Johnston, and Todd Sanger. 2019. Predicting heart failure read-mission from clinical notes using deep learning.

# SETUP

Dependencies:

In [ ]:
# pip install tensorflow==2.12.*
# pip install keras
# pip install nltk
# pip install -U scikit-learn
# pip install pandas
# pip install numpy
# pip install -U scikit-learn

In [1]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing
from time import time
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Data Download Instructions:

Link to dataset: https://physionet.org/content/mimiciii/1.4/

1.   Using the link to the dataset, create a physinet account.
2.   Navigate to he bottom webapge of the link to the "Files" section
3.   Complete the listed required training: "CITI Data or Specimens Only Research"
4.  Submit your training to physinet
5.  You'll recive an email that your application is approved, you can login in to phsionet and access the data.

# Download word2Vec and glove Embeddigns
Word2Vec PubMed-and-PMC-w2v.bin 3.08GB http://bio.nlplab.org/
Glove Common Crawl (42B tokens, 1.9M vocab, uncased, 300d vectors, 1.75 GB) https://github.com/stanfordnlp/GloVe



Paths to files and models

In [2]:
notes_path = '/content/drive/MyDrive/DL_Final_Project/NOTEEVENTS.csv'
admissions_path = '/content/drive/MyDrive/DL_Final_Project/ADMISSIONS.csv'
icd_codes_path = '/content/drive/MyDrive/DL_Final_Project/DIAGNOSES_ICD.csv'

#Word2Vec model used to generate embeddigns for CNN
biowordvecpath = '/content/drive/MyDrive/DL_Final_Project/PubMed-and-PMC-w2v.bin'

#glove model
glovepath = '/content/drive/MyDrive/DL_Final_Project/glove.42B.300d.txt'

In [3]:
# remove any HTML tags, non-word characters, numbers; convert all text to lowercase; remove stopwords
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    
    #remove stop words
    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]
    return ' '.join(tokens)

def create_train_and_val_sets(df_train, df_test):
  # x-train and y-train
    train_samples = df_train['text2'].tolist()
    train_labels = df_train['OUTPUT_LABEL'].tolist()

  # x-test and y-test
    val_samples = df_test['text2'].tolist()
    val_labels = df_test['OUTPUT_LABEL'].tolist()

    # get vocab list & assign an index to each vocab word in x-train
    vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
    text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
    vectorizer.adapt(text_ds)
    voc = vectorizer.get_vocabulary()
    word_index = dict(zip(voc, range(len(voc))))

    # finally vectorize x-train, x-test, y-train, y-test according to the above vectorizer
    x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
    x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

    y_train = np.array(train_labels)
    y_val = np.array(val_labels)

    return x_train, y_train, x_val, y_val, voc, word_index

def evaluate_model(model, x_test, y_test):
    Y_pred = np.argmax(model.predict(x_test),axis=1)
    y_true = y_test

    cnn_report1=classification_report(y_true,Y_pred,output_dict=True)
    df_cnn1=pd.DataFrame(cnn_report1).transpose()
    return df_cnn1

def subsample(df_train):
    pos_rows = df_train.OUTPUT_LABEL == 1
    df_train_pos = df_train.loc[pos_rows]
    df_train_neg = df_train.loc[~pos_rows]

    # # Merge the balanced data
    df_train_balanced = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state=42)], axis = 0)

    # Shuffle the order of training samples
    df_train_balanced = df_train_balanced.sample(n = len(df_train_balanced), random_state = 42).reset_index(drop=True)

    return df_train_balanced

def modify_sentence(sentence, p=0.5):
    for i in range(len(sentence)):
        if np.random.random() > p:
            try:
                syns = reloaded_word_vectors.most_similar([sentence[i]])
                syns = [x for (x, y) in syns]
                sentence[i] = np.random.choice(syns)
            except KeyError:
                pass
            
    return sentence

# Loading MIMIC-III Raw Data & Creating Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

df_add = pd.read_csv(admissions_path) # 58976 rows of data
df_notes = pd.read_csv(notes_path, dtype='unicode') # 2083180 rows of data
df_codes = pd.read_csv(icd_codes_path) # 651047 rows of data

Diagnosis dataframe

In [7]:
# including only rows of data with heart failure ICD-9 codes
hf_codes = ['39891', '40201', '40211', '40291', '40401', '40403', '40411', '40413', '40491', '40493', '4280', '4281', '42820','42821', '42822', '42823', '42830', '42831', '42832', '42833', '42840', '42841', '42842', '42843','4289']
df_codes = df_codes.loc[df_codes.ICD9_CODE.isin(hf_codes)] # 651047 -> 21274 rows of data

# list of subject_ids asociated with hf_codes
hf_pid_list = df_codes["SUBJECT_ID"].tolist() 

Admissions dataframe

In [12]:
# change to standard datetime format
df_add.ADMITTIME = pd.to_datetime(df_add.ADMITTIME)
df_add.DISCHTIME = pd.to_datetime(df_add.DISCHTIME)

# remove elective admissions- we only want urgent and emergency
df_adm = df_add.loc[df_add.ADMISSION_TYPE != 'ELECTIVE']

# sort by subject id and admittime
df_adm = df_add.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)

# add a column for next admit_time (readmissions) and readmission id
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
df_adm['NEXT_HADM_ID'] = df_adm.groupby('SUBJECT_ID').HADM_ID.shift(-1)
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])

# Back fill. This will take a little while.
df_adm[['NEXT_ADMITTIME','NEXT_HADM_ID']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_HADM_ID']].fillna(method = 'bfill')
df_adm['DAYS_TIL_NEXT_ADMIT'] = (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)

Clinical Notes dataframe

In [13]:
# Choosing only discharge summary clinical notes
df_notes_dis_sum = df_notes.loc[df_notes.CATEGORY == 'Discharge summary'] # 2083180 -> 59652; 

# changing type to ints so it aligns with the datatype of the other dataframes
df_notes_dis_sum['SUBJECT_ID'] = df_notes_dis_sum['SUBJECT_ID'].astype(int)
df_notes_dis_sum['HADM_ID'] = df_notes_dis_sum['HADM_ID'].astype(int)

# selecting the last discharge summary for each admission, if there are multiple
df_notes_dis_sum_last = (df_notes_dis_sum.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()

<ipython-input-13-778d94a3f7a5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notes_dis_sum['SUBJECT_ID'] = df_notes_dis_sum['SUBJECT_ID'].astype(int)
<ipython-input-13-778d94a3f7a5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notes_dis_sum['HADM_ID'] = df_notes_dis_sum['HADM_ID'].astype(int)


Merging Clinical Notes, Admissions, and Diagnoses Codes

In [14]:
# first selecting admissions for subjects that have hf
df_hf_adm = df_adm.loc[df_adm.SUBJECT_ID.isin(hf_pid_list)] # now 58976 -> 51113 -> 45321 -> 14746 rows of data

# concatenating ICD-9 codes for patient admissions with multiple hf diagnoses
df_subj_concat_icd_codes = df_codes[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']].copy()
df_subj_concat_icd_codes = df_subj_concat_icd_codes.groupby(['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].agg(' '.join).reset_index() # # 651047 -> 21274 -> 14040 rows of data

# merge the admissions and icd9-codes tables to get admissions involving hf diagnoses
df_hf_admissions = pd.merge(df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','ADMISSION_TYPE','DEATHTIME', 'NEXT_ADMITTIME', 'NEXT_HADM_ID', 'DAYS_TIL_NEXT_ADMIT']],
                        df_subj_concat_icd_codes, 
                        on = ['SUBJECT_ID', 'HADM_ID'],
                        how = 'inner')
# merge the admissions+icd-9 codes table with the discharge sumaries
df_hf_adm_notes = pd.merge(df_hf_admissions[['SUBJECT_ID','HADM_ID', 'NEXT_ADMITTIME', 'NEXT_HADM_ID', 'DAYS_TIL_NEXT_ADMIT']], df_notes_dis_sum_last, 
                        on = ['SUBJECT_ID', 'HADM_ID'],
                        how = 'inner')

# finally, create output labels for 30-day readmission; 0 for no 30-day readmission, 1 for 30-day readmission
df_hf_adm_notes['OUTPUT_LABEL'] = (df_hf_adm_notes.DAYS_TIL_NEXT_ADMIT < 30).astype('int') # consists of ____ 30-day readmission rows, and ___ without 30-day readmission rows


df_hf_adm_notes['id'] = df_hf_adm_notes.index

# shuffle input
df_adm_notes_merged = df_hf_adm_notes.sample(n=len(df_hf_adm_notes), random_state=42)
df_adm_notes_merged = df_adm_notes_merged.reset_index(drop=True)

# finalized dataset
df_final = df_adm_notes_merged.copy(deep=False) # 13755 records


# Preprocessing

Preprocessing text to remove any HTML tags, non-word characters, numbers; convert all text to lowercase; remove stopwords

In [18]:
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

stop_words = set(stopwords.words('english'))

t0 = time()
# Create new column with processed text
df_final['text2'] = df_final['TEXT'].apply(preprocessor)
elapsed=time() - t0

print("Time taken for preprocessing: ", elapsed, "seconds.")


# Split into test and train data
df_test = df_final.sample(frac=0.2, random_state=42) # 2751
df_train = df_final.drop(df_test.index) # 1104

Time taken for preprocessing:  73.49632239341736 seconds.


In [19]:
df_test = df_final.sample(frac=0.1, random_state=42) # 2751
df_train = df_final.drop(df_test.index) # 1104

# CNN

Preprocessing

In [20]:
# vectorize text to input through Word2Vec
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization

# splitting data 
x_train, y_train, x_val, y_val, voc,  word_index = create_train_and_val_sets(df_train, df_test)

In [21]:
# import pretrained embeddings  
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import numpy as np  # Make sure that numpy is imported
from nltk.corpus import stopwords

#loading pre-trained BioWordVec
reloaded_word_vectors = KeyedVectors.load_word2vec_format(biowordvecpath, binary=True)

# remove stopwords from vectors since we removed them from our data
STOPWORDS_WORD2VEC = stopwords.words('english') 
keys_updated = [word for word in reloaded_word_vectors.key_to_index if word not in STOPWORDS_WORD2VEC]
index2word_set=set(keys_updated)

In [22]:
num_tokens = len(voc) + 2
embedding_dim = 200
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    if word in index2word_set: 
        embedding_matrix[i] =  reloaded_word_vectors[word]
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19697 words (303 misses)


Training

In [23]:
embedding_layer = Embedding(
      num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,)
  
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)

In [25]:
from tensorflow.keras.optimizers import Adam
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_val, y_val))

Epoch 1/15
86/86 [==============================] - 16s 21ms/step - loss: 0.3088 - acc: 0.9146 - val_loss: 0.2871 - val_acc: 0.9189
Epoch 2/15
86/86 [==============================] - 1s 10ms/step - loss: 0.2923 - acc: 0.9163 - val_loss: 0.2835 - val_acc: 0.9189
Epoch 3/15
86/86 [==============================] - 1s 10ms/step - loss: 0.2836 - acc: 0.9163 - val_loss: 0.2843 - val_acc: 0.9189
Epoch 4/15
86/86 [==============================] - 1s 10ms/step - loss: 0.2799 - acc: 0.9163 - val_loss: 0.2825 - val_acc: 0.9189
Epoch 5/15
86/86 [==============================] - 1s 11ms/step - loss: 0.2637 - acc: 0.9163 - val_loss: 0.2912 - val_acc: 0.9189
Epoch 6/15
86/86 [==============================] - 1s 11ms/step - loss: 0.2416 - acc: 0.9165 - val_loss: 0.3012 - val_acc: 0.9189
Epoch 7/15
86/86 [==============================] - 1s 10ms/step - loss: 0.1951 - acc: 0.9250 - val_loss: 0.3502 - val_acc: 0.9168
Epoch 8/15
86/86 [==============================] - 1s 11ms/step - loss: 0.1389 - 

Evaluation

In [26]:
evaluate_model(model, x_val, y_val)

86/86 [==============================] - 0s 3ms/step


,precision,recall,f1-score,support
0,0.920163,0.980222,0.949243,2528.000000
1,0.137931,0.035874,0.056940,223.000000
accuracy,0.903671,0.903671,0.903671,0.903671
macro avg,0.529047,0.508048,0.503091,2751.000000
weighted avg,0.856755,0.903671,0.876912,2751.000000


# Random Forest

In [27]:
# for processing
import re
import nltk
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# get target values
y_train = df_train["OUTPUT_LABEL"].values
y_test = df_test["OUTPUT_LABEL"].values
X_train = df_train["text2"]
X_test = df_test["text2"]

t0 = time()

# transform training data into tf-idf vector - takes 1 minute to run
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))
corpus = X_train # make sure using the right train data
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
x_test = vectorizer.transform(X_test)
dic_vocabulary = vectorizer.vocabulary_

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
predicted = rf.predict(x_test)


elapsed=time() - t0
print("Time taken for RF training: ", elapsed, "seconds.")

report = classification_report(y_test, predicted, output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_latex()
df


Time taken for RF training:  117.63636875152588 seconds.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-27-40e4bb9ff380>:36: FutureWarning: In future ver

,precision,recall,f1-score,support
0,0.918939,1.000000,0.957757,2528.000000
1,0.000000,0.000000,0.000000,223.000000
accuracy,0.918939,0.918939,0.918939,0.918939
macro avg,0.459469,0.500000,0.478879,2751.000000
weighted avg,0.844448,0.918939,0.880120,2751.000000


# CNN - with data balancing

In [28]:
# resplitting our test and train data so that train data isn't too small when we subsample
df_test = df_final.sample(frac=0.1, random_state=42) # 1376
df_train = df_final.drop(df_test.index) # 12379

In [29]:
print("Training Data set prevalence (n = {:d}):".format(len(df_train)), "{:.2f}%".format((df_train.OUTPUT_LABEL.sum()/len(df_train))*100))

Training Data set prevalence (n = 12379): 8.43%


In [30]:
# subsampling the negatives

print("Before subsampling: ")

print("Data set prevalence (n = {:d}):".format(len(df_final)), "{:.2f}%".format((df_final.OUTPUT_LABEL.sum()/len(df_final))*100))

print("Training Data set prevalence (n = {:d}):".format(len(df_train)), "{:.2f}%".format((df_train.OUTPUT_LABEL.sum()/len(df_train))*100))

print("Test Data set prevalence (n = {:d}):".format(len(df_test)), "{:.2f}%".format((df_test.OUTPUT_LABEL.sum()/len(df_test))*100))

df_train_balanced = subsample(df_train)

print("After subsampling: ")

print("Training Data set prevalence (n = {:d}):".format(len(df_train_balanced)), "{:.2f}%".format((df_train_balanced.OUTPUT_LABEL.sum()/len(df_train_balanced))*100))

Before subsampling: 
Data set prevalence (n = 13755): 8.32%
Training Data set prevalence (n = 12379): 8.43%
Test Data set prevalence (n = 1376): 7.27%
After subsampling: 
Training Data set prevalence (n = 2088): 50.00%


In [31]:
x_train, y_train, x_val, y_val, voc,  word_index = create_train_and_val_sets(df_train_balanced, df_test)

In [32]:
num_tokens = len(voc) + 2
embedding_dim = 200
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    if word in index2word_set: 
        embedding_matrix[i] =  reloaded_word_vectors[word]
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 18495 words (1505 misses)


In [33]:

from tensorflow.keras.optimizers import Adam

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_val, y_val))

evaluate_model(model, x_val, y_val)

Epoch 1/15
17/17 [==============================] - 3s 48ms/step - loss: 0.7449 - acc: 0.5086 - val_loss: 0.7020 - val_acc: 0.2413
Epoch 2/15
17/17 [==============================] - 0s 14ms/step - loss: 0.6969 - acc: 0.5144 - val_loss: 0.7263 - val_acc: 0.0741
Epoch 3/15
17/17 [==============================] - 0s 13ms/step - loss: 0.6930 - acc: 0.5230 - val_loss: 0.7299 - val_acc: 0.0727
Epoch 4/15
17/17 [==============================] - 0s 14ms/step - loss: 0.6886 - acc: 0.5441 - val_loss: 0.6407 - val_acc: 0.8728
Epoch 5/15
17/17 [==============================] - 0s 14ms/step - loss: 0.6772 - acc: 0.5872 - val_loss: 0.6149 - val_acc: 0.8445
Epoch 6/15
17/17 [==============================] - 0s 15ms/step - loss: 0.6713 - acc: 0.5996 - val_loss: 0.6212 - val_acc: 0.7304
Epoch 7/15
17/17 [==============================] - 0s 13ms/step - loss: 0.6934 - acc: 0.5086 - val_loss: 0.6481 - val_acc: 0.8852
Epoch 8/15
17/17 [==============================] - 0s 13ms/step - loss: 0.6806 - a

,precision,recall,f1-score,support
0,0.935857,0.697492,0.799282,1276.000000
1,0.091765,0.390000,0.148571,100.000000
accuracy,0.675145,0.675145,0.675145,0.675145
macro avg,0.513811,0.543746,0.473926,1376.000000
weighted avg,0.874513,0.675145,0.751992,1376.000000


# Random Forest - with data balancing

In [34]:
# for processing
import re
import nltk
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# get target values
y_train = df_train_balanced["OUTPUT_LABEL"].values
y_test = df_test["OUTPUT_LABEL"].values
X_train = df_train_balanced["text2"]
X_test = df_test["text2"]


# transform training data into tf-idf vector - takes 1 minute to run
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))
corpus = X_train # make sure using the right train data
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
x_test = vectorizer.transform(X_test)
dic_vocabulary = vectorizer.vocabulary_

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
predicted = rf.predict(x_test)

report = classification_report(y_test, predicted, output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_latex()
df


<ipython-input-34-6412ff0865f8>:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex()


,precision,recall,f1-score,support
0,0.958128,0.609718,0.745211,1276.000000
1,0.117021,0.660000,0.198795,100.000000
accuracy,0.613372,0.613372,0.613372,0.613372
macro avg,0.537575,0.634859,0.472003,1376.000000
weighted avg,0.897001,0.613372,0.705500,1376.000000


# Main Experiment: CNN - with data augmentation and balancing

In [35]:
print("Training Data set prevalence (n = {:d}):".format(len(df_train)), "{:.2f}%".format((df_train.OUTPUT_LABEL.sum()/len(df_train))*100)) # should have gone up from 8% to 11% after data aug

Training Data set prevalence (n = 12379): 8.43%


In [36]:

pos_rows = df_train.OUTPUT_LABEL == 1
df_pos = df_train.loc[pos_rows]
df_neg = df_train.loc[~pos_rows]

df_pos = df_pos.reset_index(drop=True)

df_pos['id2'] = df_pos.index

indexes = np.random.randint(0, df_pos.shape[0], 1000)

for num, i in enumerate(indexes):
    x = df_pos.loc[df_pos.id2==i]['text2']
    sample =  str(np.trim_zeros(x)).split()
    if str(i) in sample:
        sample.remove(str(i))

    modified = modify_sentence(sample)
    sentence_m = ' '.join([x for x in modified])

    df2 = pd.DataFrame({'text2': [sentence_m], 'OUTPUT_LABEL': [1]})
    df_pos = pd.concat([df2, df_pos], ignore_index = True)
    df_pos.reset_index()

df_train_aug = pd.concat([df_pos, df_neg], axis = 0)

In [37]:
print("Before subsampling: ")

print("Training Data set prevalence (n = {:d}):".format(len(df_train_aug)), "{:.2f}%".format((df_train_aug.OUTPUT_LABEL.sum()/len(df_train_aug))*100))

df_train_balanced_and_aug = subsample(df_train_aug)

print("After subsampling: ")

print("Training Data set prevalence (n = {:d}):".format(len(df_train_balanced_and_aug)), "{:.2f}%".format((df_train_balanced_and_aug.OUTPUT_LABEL.sum()/len(df_train_balanced_and_aug))*100))

Before subsampling: 
Training Data set prevalence (n = 13379): 15.28%
After subsampling: 
Training Data set prevalence (n = 4088): 50.00%


In [40]:
x_train, y_train, x_val, y_val, voc,  word_index = create_train_and_val_sets(df_train_balanced_and_aug, df_test)

In [41]:
num_tokens = len(voc) + 2
embedding_dim = 200
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    if word in index2word_set: 
        embedding_matrix[i] =  reloaded_word_vectors[word]
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19221 words (779 misses)


In [42]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)

x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)

x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
# model.summary()
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_val, y_val))

evaluate_model(model, x_val, y_val)

Epoch 1/15
32/32 [==============================] - 3s 32ms/step - loss: 0.6088 - acc: 0.7128 - val_loss: 0.5470 - val_acc: 0.8895
Epoch 2/15
32/32 [==============================] - 1s 19ms/step - loss: 0.4902 - acc: 0.7397 - val_loss: 0.5253 - val_acc: 0.9230
Epoch 3/15
32/32 [==============================] - 0s 13ms/step - loss: 0.4766 - acc: 0.7441 - val_loss: 0.4587 - val_acc: 0.9251
Epoch 4/15
32/32 [==============================] - 0s 12ms/step - loss: 0.4679 - acc: 0.7427 - val_loss: 0.5061 - val_acc: 0.9106
Epoch 5/15
32/32 [==============================] - 0s 12ms/step - loss: 0.4543 - acc: 0.7532 - val_loss: 0.3462 - val_acc: 0.9273
Epoch 6/15
32/32 [==============================] - 0s 11ms/step - loss: 0.4215 - acc: 0.7828 - val_loss: 0.4076 - val_acc: 0.9026
Epoch 7/15
32/32 [==============================] - 0s 11ms/step - loss: 0.3581 - acc: 0.8293 - val_loss: 0.4934 - val_acc: 0.8517
Epoch 8/15
32/32 [==============================] - 0s 12ms/step - loss: 0.2682 - a

,precision,recall,f1-score,support
0,0.932261,0.862853,0.896215,1276.00000
1,0.102564,0.200000,0.135593,100.00000
accuracy,0.814680,0.814680,0.814680,0.81468
macro avg,0.517412,0.531426,0.515904,1376.00000
weighted avg,0.871963,0.814680,0.840937,1376.00000


# Main Experiment: Random Forest - with data augmentation and balancing

In [43]:
# # ## get target values
Y_train = df_train_balanced_and_aug["OUTPUT_LABEL"].values
Y_test = df_test["OUTPUT_LABEL"].values
X_train = df_train_balanced_and_aug["text2"]
X_test = df_test["text2"]


# transform training data into tf-idf vector - takes 1 minute to run
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))
corpus = X_train # make sure using the right train data
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
X_test = vectorizer.transform(X_test)
dic_vocabulary = vectorizer.vocabulary_

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
predicted = rf.predict(X_test)

report = classification_report(Y_test, predicted, output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_latex()
df

<ipython-input-43-5ef535c23c5e>:22: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex()


,precision,recall,f1-score,support
0,0.930268,0.951411,0.940721,1276.000000
1,0.126761,0.090000,0.105263,100.000000
accuracy,0.888808,0.888808,0.888808,0.888808
macro avg,0.528514,0.520705,0.522992,1376.000000
weighted avg,0.871874,0.888808,0.880004,1376.000000


# Ablation 1: CNN + Global average pooling vs Max pooling

In [44]:
# from keras.layers import GlobalAveragePooling1D
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)

x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)

x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 200)         4000400   
                                                                 
 conv1d_9 (Conv1D)           (None, None, 128)         128128    
                                                                 
 max_pooling1d_6 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, None, 128)         82048     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, None, 128)        0         
 1D)                                                       

In [46]:
x_train, y_train, x_val, y_val, voc,  word_index = create_train_and_val_sets(df_train_balanced_and_aug, df_test)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_val, y_val))

evaluate_model(model, x_val, y_val)

Epoch 1/15
32/32 [==============================] - 2s 20ms/step - loss: 0.0367 - acc: 0.9865 - val_loss: 1.1361 - val_acc: 0.7267
Epoch 2/15
32/32 [==============================] - 0s 12ms/step - loss: 0.0107 - acc: 0.9956 - val_loss: 1.5195 - val_acc: 0.6926
Epoch 3/15
32/32 [==============================] - 0s 12ms/step - loss: 0.0181 - acc: 0.9932 - val_loss: 0.9796 - val_acc: 0.7762
Epoch 4/15
32/32 [==============================] - 0s 14ms/step - loss: 0.0133 - acc: 0.9973 - val_loss: 1.8485 - val_acc: 0.6221
Epoch 5/15
32/32 [==============================] - 0s 13ms/step - loss: 0.0171 - acc: 0.9954 - val_loss: 0.7028 - val_acc: 0.8314
Epoch 6/15
32/32 [==============================] - 0s 14ms/step - loss: 0.0146 - acc: 0.9956 - val_loss: 1.4374 - val_acc: 0.6890
Epoch 7/15
32/32 [==============================] - 0s 14ms/step - loss: 0.0167 - acc: 0.9951 - val_loss: 1.1182 - val_acc: 0.7420
Epoch 8/15
32/32 [==============================] - 0s 14ms/step - loss: 0.0197 - a

,precision,recall,f1-score,support
0,0.930958,0.655172,0.769089,1276.000000
1,0.079498,0.380000,0.131488,100.000000
accuracy,0.635174,0.635174,0.635174,0.635174
macro avg,0.505228,0.517586,0.450289,1376.000000
weighted avg,0.869078,0.635174,0.722752,1376.000000


# Ablation 2: Comparing GloVe to pre-trained Word2Vec

In [47]:
#loading glove model
embeddings_dict = {}

with open(glovepath, 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector


In [48]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    if word in embeddings_dict:
        embedding_matrix[i] =  embeddings_dict[word]
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17296 words (2704 misses)


In [49]:
embedding_layer = Embedding(
      num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,)
  
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)

In [50]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_data=(x_val, y_val))

evaluate_model(model, x_val, y_val)

Epoch 1/15
32/32 [==============================] - 5s 30ms/step - loss: 0.5283 - acc: 0.7189 - val_loss: 0.5742 - val_acc: 0.9266
Epoch 2/15
32/32 [==============================] - 0s 14ms/step - loss: 0.4864 - acc: 0.7432 - val_loss: 0.4799 - val_acc: 0.9273
Epoch 3/15
32/32 [==============================] - 0s 14ms/step - loss: 0.4727 - acc: 0.7466 - val_loss: 0.5706 - val_acc: 0.9251
Epoch 4/15
32/32 [==============================] - 0s 15ms/step - loss: 0.4552 - acc: 0.7500 - val_loss: 0.5341 - val_acc: 0.8372
Epoch 5/15
32/32 [==============================] - 0s 14ms/step - loss: 0.3997 - acc: 0.8082 - val_loss: 0.4861 - val_acc: 0.7907
Epoch 6/15
32/32 [==============================] - 0s 14ms/step - loss: 0.2223 - acc: 0.9161 - val_loss: 0.7429 - val_acc: 0.6047
Epoch 7/15
32/32 [==============================] - 0s 14ms/step - loss: 0.0995 - acc: 0.9667 - val_loss: 0.9181 - val_acc: 0.5763
Epoch 8/15
32/32 [==============================] - 0s 14ms/step - loss: 0.0475 - a

,precision,recall,f1-score,support
0,0.927495,0.771944,0.842601,1276.000000
1,0.073248,0.230000,0.111111,100.000000
accuracy,0.732558,0.732558,0.732558,0.732558
macro avg,0.500372,0.500972,0.476856,1376.000000
weighted avg,0.865413,0.732558,0.789440,1376.000000
